In [7]:
from parler import Parler
import pandas as pd
from tqdm import tqdm

### Top 20 hashtags

The following requires the user to enter a search term and returns the top 20 hashtags sorted by volume

In [ ]:
q = input('Enter search term: ')

dict1 = {}
dict1['tag'] = []
dict1['totalPosts'] = []

print('Collecting hashtags that contain ' + q + '..')

fetch = Parler().getHashtags(q)

for x in fetch['tags']:
    dict1['tag'].append(x['tag'])
    dict1['totalPosts'].append(x['totalPosts'])

df_tags = pd.DataFrame.from_dict(dict1)

print(df_tags)

### Top 10 affiliate domains

The following fetches the domains for a specified amount of pages (in this case 5) of affiliate news posts. Just change the "5" to a higher number to increase the number of posts you want to collect. 

In [ ]:
dict1 = {}
dict1["domain"] = []

fetch = Parler().getAffiliateFeed()
next_cursor = fetch["next"]

# There are a lot of domains named "feedproxy.google.com", so I added a few lines to get the site if this is the domain
for x in fetch["links"]:
    if x["domain"] != "feedproxy.google.com":
        dict1["domain"].append(x["domain"])
    elif x["domain"] == "feedproxy.google.com":
        dict1["domain"].append(x["metadata"]["site"])

# Change "5" to a higher number to collect more posts
for i in tqdm(range(1,5),desc="Collecting affiliate data..."):
    fetch = Parler().getAffiliateFeed(cursor=next_cursor)
    next_cursor = fetch["next"]
    for x in fetch["links"]:
        if x["domain"] != "feedproxy.google.com":
            dict1["domain"].append(x["domain"])
        elif x["domain"] == "feedproxy.google.com":
            dict1["domain"].append(x["metadata"]["site"])

df_affiliate_data = pd.DataFrame.from_dict(dict1)

# Return 10 most frequent domains
most_freq = pd.Series(' '.join(df_affiliate_data["domain"]).lower().split()).value_counts()[:10]
print(most_freq)

### Retrieve all posts which contain a specified hashtag

The following pages through and collects all posts which contain a specified hashtag and puts the id of the post and body of the post into a data frame for manipulation later

In [ ]:
hashtag = input('Enter hashtag: ')

dict1 = {}
dict1["id"] = []
dict1["body"] = []

df_hashtag_data = pd.DataFrame()
    
print("Collecting data for #" + hashtag + "...")
    
fetch = Parler().getHashtagFeed(hashtag)
next_cursor = fetch["next"]
    
for x in fetch["posts"]:
    dict1["id"].append(x["id"])
    dict1["body"].append(x["body"])

while fetch["last"] == False:
    fetch = Parler().getHashtagFeed(hashtag,cursor=next_cursor)
    next_cursor = fetch["next"]
    for x in fetch["posts"]:
        dict1["id"].append(x["id"])
        dict1["body"].append(x["body"])
    
df_hashtag_data = pd.DataFrame.from_dict(dict1)
df_hashtag_data